## Importando Bibliotecas

In [3]:
import pandas as pd
import numpy as np

## Importando base de dados

In [6]:
#load data from txt file with no head witch the data is splot by :::
data = pd.read_csv('Classification_Dataset/train_data.txt', sep=':::', engine='python',header=None)
#remove index 
data = data.drop([0], axis=1)
#set dataframe head 
data.columns = ['name', 'genre','description']

                                 name       genre  \
0       Oscar et la dame rose (2009)       drama    
1                       Cupid (1997)    thriller    
2   Young, Wild and Wonderful (1980)       adult    
3              The Secret Sin (1915)       drama    
4             The Unrecovered (2007)       drama    

                                         description  
0   Listening in to a conversation between his do...  
1   A brother and sister with a past incestuous r...  
2   As the bus empties the students for their fie...  
3   To help their unemployed father make ends mee...  
4   The film's title refers not only to the un-re...  


## ETL

In [7]:
#removing whitespaces from the start and end of the colum "genre"
data['genre'] = data['genre'].str.strip()

In [8]:
print("before filtering genres: ",len(data))
filter_list = ['drama', 'comedy', 'horror', 'thriller', 'action', 'mystery', 'sci-fi', 'fantasy', 'western', 'romance', 'music']
data = data[data['genre'].isin(filter_list)]
print("after filtering genres: ",len(data))

before filtering genres:  54214
after filtering genres:  29894


Vamo ver se a gente vai usar isso, talvez tentar primeiro sem:

In [9]:
#creating a "super-genre" out of "horror", "thriller" and "mystery" 
data['genre'] = data['genre'].replace(['horror', 'thriller', 'mystery'], 'h-t-m')

#creating a "super-genre" out of "sci-fi" and "fantasy"
data['genre'] = data['genre'].replace(['sci-fi', 'fantasy'], 's-f')

## EDA

In [10]:
#ploting the number of movies in each genre
data['genre'].value_counts().plot(kind='bar', title='Number of movies in each genre')

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.